Practical 9 : Implement text processing with neural network

In [6]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import matplotlib.pyplot as plt

In [7]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [8]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to C:\Users\Anisha\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [9]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


In [10]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'WHEN FRIENDSHIP KILLS, in my opinion, is a very touching and kind of heartbreaking drama about the consequences of being anorexic or bulimic. Anytime Lexi (Katie Wright) or Jennifer (Marley Shelton) threw up, I wanted to vomit myself. It\'s kind of hard to explain why. If you ask me, they should have been more cooperative about things. However, I did enjoy seeing them do things together as well as get lectured by their parents. Before I wrap this up, I\'d like to say, "If you ask me, WHEN FRIENDSHIP KILLS does indeed show you how being anorexic or bulimic can affect a person\'s body. " Now, in conclusion, I recommend this movie to everyone who hasn\'t seen it. You\'re in for some tears and a good time, so the next time it\'s on TV, kick back with a friend and watch it.'
 b"Almost a two-person play, and as such the dialog and the performances of the leads will be important. Neither are particularly good. This might have been stronger, in fact, if it had first been crafted as 

In [12]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [13]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 51,   1,   1, ...,   0,   0,   0],
       [210,   4,   1, ...,   0,   0,   0],
       [ 49,   4,   1, ...,   0,   0,   0]], dtype=int64)

In [14]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'WHEN FRIENDSHIP KILLS, in my opinion, is a very touching and kind of heartbreaking drama about the consequences of being anorexic or bulimic. Anytime Lexi (Katie Wright) or Jennifer (Marley Shelton) threw up, I wanted to vomit myself. It\'s kind of hard to explain why. If you ask me, they should have been more cooperative about things. However, I did enjoy seeing them do things together as well as get lectured by their parents. Before I wrap this up, I\'d like to say, "If you ask me, WHEN FRIENDSHIP KILLS does indeed show you how being anorexic or bulimic can affect a person\'s body. " Now, in conclusion, I recommend this movie to everyone who hasn\'t seen it. You\'re in for some tears and a good time, so the next time it\'s on TV, kick back with a friend and watch it.'
Round-trip:  when [UNK] [UNK] in my opinion is a very [UNK] and kind of [UNK] drama about the [UNK] of being [UNK] or [UNK] [UNK] [UNK] [UNK] [UNK] or [UNK] [UNK] [UNK] [UNK] up i wanted to [UNK] myself its

In [15]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [16]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [17]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.01367362]


In [18]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.01367362]


In [19]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset,
                    validation_steps=30)

test_loss, test_acc = model.evaluate(test_dataset)

Epoch 1/3
391/391 [==============================] - 1014s 3s/step - loss: 0.6224 - accuracy: 0.5925 - val_loss: 0.4624 - val_accuracy: 0.7786
Epoch 2/3
391/391 [==============================] - 980s 3s/step - loss: 0.3892 - accuracy: 0.8216 - val_loss: 0.3637 - val_accuracy: 0.8245
Epoch 3/3
367/391 [===========================>..] - ETA: 12s - loss: 0.3346 - accuracy: 0.8392

In [ ]:
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)